In [31]:
import pandas as pd
from IPython.display import display
from datetime import datetime, timedelta
from GSPC_model import process_data_and_backtest
import yfinance as yf

pd.options.display.float_format = '{:.2f}'.format

In [32]:
def simulate_trading_algorithm(starting_balance, data, index_date):
    current_balance = starting_balance
    shares_held = 0
    cost_basis = 0

    trading_log = []

    for i, current_date in enumerate(index_date[:-1]):
        # Extract relevant data
        row = data.loc[current_date]
        prediction = row['Predictions_SP500']
        vfiax_close = row['VFIAX_close']

        # Buy
        if prediction == 1 and shares_held == 0:
            shares_bought = current_balance / vfiax_close
            cost_basis = current_balance
            shares_held += shares_bought
            current_balance = 0

            trading_log.append({
                'Date': current_date,
                'Prediction': prediction,
                'Decision': 'Buy',
                'Shares': shares_bought,
                'Price': vfiax_close,
                'Profit/Loss': 0,  
                'Current Balance': current_balance
            })

        # Hold
        elif prediction == 1 and shares_held > 0:
            current_balance = shares_held * vfiax_close

            trading_log.append({
                'Date': current_date,
                'Prediction': prediction,
                'Decision': 'Hold',
                'Shares': shares_held,
                'Price': vfiax_close,
                'Profit/Loss': 0,
                'Current Balance': current_balance
            })

        # Sell
        elif prediction == 0 and shares_held > 0:
            current_balance = shares_held * vfiax_close
            profit_loss = current_balance - cost_basis
            shares_held = 0
            cost_basis = 0

            trading_log.append({
                'Date': current_date,
                'Prediction': prediction,
                'Decision': 'Sell',
                'Shares': 0,
                'Price': vfiax_close,
                'Profit/Loss': profit_loss,
                'Current Balance': current_balance
            })

    trading_log_df = pd.concat([pd.DataFrame([x]) for x in trading_log], ignore_index=True)

    return current_balance, trading_log_df

In [33]:
vfiax = yf.Ticker("VFIAX")
vfiax = vfiax.history(start="1990-01-02", end=None)

sp500 = process_data_and_backtest()

sp500 = pd.merge(sp500, vfiax['Close'].rename('VFIAX_close'), left_index=True, right_index=True, how='left')

result, trading_log = simulate_trading_algorithm(10000, sp500, sp500.index)

print("Final balance:", result)

print("\nTrading Log:")
display(trading_log)

Precision: 0.7506963788300836
Value Counts of Predictions:
Predictions
0.00    1647
1.00     718
Name: count, dtype: int64
Final balance: 728214.1740054

Trading Log:


,Date,Prediction,Decision,Shares,Price,Profit/Loss,Current Balance
0,2014-07-29 00:00:00-04:00,1.00,Buy,64.65,154.68,0.00,0.00
1,2014-07-30 00:00:00-04:00,0.00,Sell,0.00,154.71,2.20,10002.20
2,2014-08-01 00:00:00-04:00,1.00,Buy,66.16,151.19,0.00,0.00
3,2014-08-04 00:00:00-04:00,1.00,Hold,66.16,152.28,0.00,10074.20
4,2014-08-05 00:00:00-04:00,1.00,Hold,66.16,150.82,0.00,9977.45
...,...,...,...,...,...,...,...
1085,2023-12-04 00:00:00-05:00,1.00,Buy,1702.23,422.83,0.00,0.00
1086,2023-12-05 00:00:00-05:00,0.00,Sell,0.00,422.61,-374.49,719379.34
1087,2023-12-06 00:00:00-05:00,1.00,Buy,1708.86,420.97,0.00,0.00
1088,2023-12-07 00:00:00-05:00,1.00,Hold,1708.86,424.39,0.00,725223.67
